# **METÓDOS DAS DIFERENÇAS FINITAS** 

PROBLEMAS DE CONTORNO


In [106]:
from sympy import symbols, IndexedBase, Sum, Eqn, simplify, Abs
import numpy as np
from algebra_with_sympy import *


In [107]:
i, h, z = symbols('i h z')
y = IndexedBase('y')
x = IndexedBase('x')

yll = Eqn(y[i] ,(y[i+1]-2*y[i]+y[i-1])/(h**2)) 
ylprog = Eqn(y[i],((y[i+1]-y[i])/h))
ylregr = Eqn(y[i],((y[i]-y[i-1])/h))
ylcent = Eqn(y[i],((y[i+1]-y[i-1])/(2*h)))

display(yll, ylprog, ylregr, ylcent)


Equation(y[i], (y[i + 1] + y[i - 1] - 2*y[i])/h**2)

Equation(y[i], (y[i + 1] - y[i])/h)

Equation(y[i], (-y[i - 1] + y[i])/h)

Equation(y[i], (y[i + 1] - y[i - 1])/(2*h))

**Discretizando**

In [108]:
edo = Eqn(-yll.rhs + y[i], x[i])  #pegando o lado direito da equação yll e aplicando na função
edo

Equation(y[i] - (y[i + 1] + y[i - 1] - 2*y[i])/h**2, x[i])

In [109]:
edo = simplify(edo)*h**2
edo

Equation(h**2*y[i] - y[i + 1] - y[i - 1] + 2*y[i], h**2*x[i])

In [110]:
# Agrupando os elementos em comum
common_y_terms = edo.lhs.collect(y[i])
remaining_terms = edo.rhs

# Equação discretizada final
grouped_equation = Eq(common_y_terms, remaining_terms)
grouped_equation

Eq((h**2 + 2)*y[i] - y[i + 1] - y[i - 1], h**2*x[i])

# Condição de Robin
Tenho aqui o meu vetor com h=0.25
| i |  x  | y |
| --- | --- | --- |
| 0 | 0.00 | -1  |
| 1 | 0.25 |     |
| 2 | 0.50 |     |
| 3 | 0.75 |     |
| 4 | 1.00 |     |

E, dado:
y'(0) = 1
y (0) = -1

In [111]:
#Meus vetores
y0 = 1
xv  = [0, 0.25, 0.50, 0.75, 1]
yv  = [-1, nan, nan, nan, nan]

In [112]:
n=4
eq = {}
for k in range(1,n):
    equation = grouped_equation.subs(i, k)
    eq[k] = equation
    display(eq[k])

    


Eq((h**2 + 2)*y[1] - y[0] - y[2], h**2*x[1])

Eq((h**2 + 2)*y[2] - y[1] - y[3], h**2*x[2])

Eq((h**2 + 2)*y[3] - y[2] - y[4], h**2*x[3])

Substituindo o valor de y{0}:

In [113]:
z = yv[0] #valor inicial y0 que temos
for k in range(1, len(eq)+1):
    eq[k] = eq[k].evalf(4, subs={y[0]: z})
    display(eq[k])



Eq((h**2 + 2.0)*y[1] - y[2] + 1.0, h**2*x[1])

Eq((h**2 + 2.0)*y[2] - y[1] - y[3], h**2*x[2])

Eq((h**2 + 2.0)*y[3] - y[2] - y[4], h**2*x[3])

In [114]:
#Colocando o valor y0 para o outro lado da equação
#Neste caso só preciso mudar a primeira Equação
common_y_terms = eq[1].lhs.collect(y[i])
remaining_terms = eq[1].rhs
eq[1] = Eq(common_y_terms + z, remaining_terms+z) #aqui estou jogando o valor de y0 para o outro lado
eq[1]

Eq((h**2 + 2.0)*y[1] - y[2], h**2*x[1] - 1)

In [115]:
#Se tenho o primeiro valor, então procedo com a Diferença Finita Regressiva

In [116]:
n = len(eq)+1
ylregr = ylregr.subs(i, n)
display(ylregr)
ylregr_lhs = ylregr.lhs.subs(y[n], y0)
eq_substituted = Eq(ylregr_lhs, ylregr.rhs)
eq_substituted

Equation(y[4], (-y[3] + y[4])/h)

Eq(1, (-y[3] + y[4])/h)

In [117]:
# Construindo a nova equação com h do outro lado
equation_moved = Eq(ylregr_lhs * h, ylregr.rhs * h)
equation_moved

Eq(h, -y[3] + y[4])

In [118]:
# Movendo y[i-1] para o lado esquerdo da equação
eq_moved = Eq(equation_moved.rhs + y[n-1],equation_moved.lhs+y[n-1])
eq_moved

Eq(y[4], h + y[3])

In [119]:
#substituindo na ultima equacao
eqmoved = Eq(eq[n-1].lhs - eq_moved.rhs + eq_moved.lhs, eq[n-1].rhs)
eqmoved

Eq(-h + (h**2 + 2.0)*y[3] - y[2] - y[3], h**2*x[3])

In [120]:
# Agrupando os elementos em comum
common_y_terms = eqmoved.lhs.collect(y[n-1])
remaining_terms = eqmoved.rhs

# Equação discretizada final
grouped_equation = Eq(common_y_terms, remaining_terms)
grouped_equation

Eq(-h + (h**2 + 1.0)*y[3] - y[2], h**2*x[3])

In [121]:
#movendo o h para o outro lado e finalizando a ultima equação
const = -1
eq[n-1] = Eq(grouped_equation.lhs - const*h, grouped_equation.rhs + h) 
eq[n-1] 

Eq((h**2 + 1.0)*y[3] - y[2], h**2*x[3] + h)

In [122]:
for k in range(1, len(eq)+1):
    display(eq[k])


Eq((h**2 + 2.0)*y[1] - y[2], h**2*x[1] - 1)

Eq((h**2 + 2.0)*y[2] - y[1] - y[3], h**2*x[2])

Eq((h**2 + 1.0)*y[3] - y[2], h**2*x[3] + h)

In [123]:
matrix = [[0 for _ in range(n-1)] for _ in range(n-1)]
matrix
for i in range(1,n):
   # print("Eq ", i, "---------------")
    for j in range(len(eq[i].args[0].args)): #len(eq[2].args[0].args) é a qtd de y que tenho
    #    print(eq[i].args[0].args[j])
        if eq[i].args[0].args[j].args[1].args[1] == 1:
            matrix[i-1][0] = eq[i].args[0].args[j].args[0]
        if eq[i].args[0].args[j].args[1].args[1] == 2:
            matrix[i-1][1] = eq[i].args[0].args[j].args[0]
        if eq[i].args[0].args[j].args[1].args[1] == 3:
            matrix[i-1][2] = eq[i].args[0].args[j].args[0]

left = []
for i in range(1,n):
    left.append(eq[i].rhs)
    

print("Minha Matriz:")
display(Matrix(matrix))
display(Matrix(left))

Minha Matriz:


Matrix([
[h**2 + 2.0,         -1,          0],
[        -1, h**2 + 2.0,         -1],
[         0,         -1, h**2 + 1.0]])

Matrix([
[h**2*x[1] - 1],
[    h**2*x[2]],
[h**2*x[3] + h]])

In [124]:
A = np.array(matrix.copy())
for i in range(len(matrix)):
    A[i, i] = A[i, i].subs(h, 0.25)

B = np.array(left.copy())
for i in range(len(matrix)):
    B[i] = B[i].subs(h, 0.25)
    B[i] = B[i].subs(x[i+1],xv[i+1])


    
print("Meu Sistema:")
display(Matrix(A))
display(Matrix(B))

Meu Sistema:


Matrix([
[2.0625,     -1,      0],
[    -1, 2.0625,     -1],
[     0,     -1, 1.0625]])

Matrix([
[-0.984375],
[  0.03125],
[ 0.296875]])

In [125]:
matrix_aumentada = np.concatenate((Matrix(A),Matrix(B)), axis=1)
print("Matriz Aumentada:")
Matrix(matrix_aumentada)

Matriz Aumentada:


Matrix([
[2.0625,     -1,      0, -0.984375],
[    -1, 2.0625,     -1,   0.03125],
[     0,     -1, 1.0625,  0.296875]])

In [126]:
from thomas_system_solve import thomas
thomas(matrix_aumentada)

Solução
x 0 = -0.4581
x 1 = 0.0396
x 2 = 0.0
